- handle different variations of the same brand: 1 vs. ugg, doc vs. dr
- test string being matched to a sentence
- speed issue
- handle multiple choices in one candidate: 'Adidas and hey dude', ' Converse high top and vans'
- handle nan

In [13]:
import sys

if "../../src/" not in sys.path:
    sys.path.append("../../src/")
    
import numpy as np
import pandas as pd
from pathlib import Path
from pprint import pprint

from fuzzywuzzy import fuzz, process
from match import Matcher

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [14]:
path = Path.cwd().parent.parent / 'data' / 'raw' / 'journeys.csv'
df = pd.read_csv(path)
df.shape
df.head()

(10127, 5)

,question_id,reply_id,created_at,question,reply
0,31665,18475753,2020-11-27 18:34:35,Tell us...what brand(s) did you purchase?,Uggs
1,31665,21873401,2021-02-05 17:24:45,Tell us...what brand(s) did you purchase?,Adidas
2,31665,21871152,2021-02-05 16:43:52,Tell us...what brand(s) did you purchase?,Timberland
3,31665,21873596,2021-02-05 17:28:04,Tell us...what brand(s) did you purchase?,Converse
4,31665,21871473,2021-02-05 16:52:05,Tell us...what brand(s) did you purchase?,Red tims


In [15]:
#df.question_id.unique()

In [16]:
df_31665 = df.query('question_id==31665')
df_31665.shape

strings_to_match = df_31665.reply.values
strings_to_match

(8497, 5)

array(['Uggs', 'Adidas ', 'Timberland', ..., '5',
       'Adidas and Berkenstock', 'Doc Martens'], dtype=object)

In [17]:
#num_uniques = len(df_31665.reply.unique())
#print(f'The nunmber of unique replies is {num_uniques}')

#df_31665.reply.value_counts()

In [18]:
process.extractOne(query='i love ughs', choices=['ugg', 'ugh'])

('ugh', 90)

In [19]:
patterns = [
    'vans',
    'converse',
    'dr. martens',
    'uggs',
    'adidas',
    'crocs',
    'timberlands',
    'journey\'s',
    'dude',
    'new balance',
    'fila',
    'puma',
    'sperry',
    'birkenstock',
    'steve madden',
    'north face',
    'toms',
    'dc',
    'reebok',
    'roxy',
    'champion',
    'jan sport',
    'heelys',
    'polo',
    'nike',
    'kahken',
    'teva',
    'sorel',
    'tommy hilfiger',
    'herschel',
    'lacoste',
    'perry',
    'clarks',
    'speedy',
    'rainbow',
    'ralph lauren',
    'reefs',
    'levi\'s',
    'chaco',
    'madden girl'    
]

patterns = sorted(patterns)
#patterns

In [20]:
m = Matcher(choices=patterns)

In [21]:
m.threshold

80

In [22]:
results = m.run(queries=strings_to_match, limit=1)
pprint(results)

{nan: 'no_match',
 '\nAdidas': 'adidas',
 '   Vans ': 'vans',
 '  Vans ': 'vans',
 ' Converse high top and vans': 'converse',
 ' Crocs': 'crocs',
 ' Crocs ': 'crocs',
 ' Timberlands': 'timberlands',
 ' Vans': 'vans',
 ' Vans ': 'vans',
 ' Vans and converse ': 'converse',
 ' White doc martens': 'dr. martens',
 " ugg's ": 'uggs',
 '1 Vans 2 Converse & 1 Filaz': 'converse',
 '1 converse ': 'converse',
 '1 doc martene': 'no_match',
 "1 park of ugg's and 2 pairs of ugg's ": "journey's",
 '1 vans': 'vans',
 '1 vans ': 'vans',
 '2': 'no_match',
 '2 dragon ball z t shirts': 'no_match',
 '2 packs of low cut Journeys socks and a champions headwear': 'champion',
 '2 pair of Adidas and one pair of vans': 'adidas',
 "2 pair of Dr. Martens... 1 pair of women's platform 1 pair of men boots ": 'dr. '
                                                                             'martens',
 '2 pair of Pumas 1 Filas 1 Converse 1 Herschel Fanny Pack 1 set of no show Journeys socks 1 set of super soft Journ

In [23]:
m.report()

-------------------Matching Report-------------------
Using threshold of 80
9.31% of the replies couldn't be matched (791/8497). Samples:
array(['UGGA', 'Fifa socks and exchange of vans shoe', 'Yes', 'test',
       'Chelsea ', 'Vqns', 'Yuh', 'Doc Martin',
       'Vans, Rick and Morty, Metallic, Simpsons ', 'Ugh',
       'Doctor martins ', 'B52', 'fjallraven kanken ', 'Ugh', 'Birks ',
       'Ugg Cleaner', 'Ugh', 'Hydro', 'Socks and boots\n', 'Tee shirt '],
      dtype=object)
-------------------End of Report-------------------


In [28]:
queries=['Hydro flask', '2 dragon ball z t shirts', 'test',
         '2976 Chelsea ', 'mia', 'Shoe cleaner ', '3/3', 'AS OX OCEAN BLISS\n&\nSOFTEY PNK',
         "Vans. They weren't in stock so I ordered and shipped to my house ",
        'Removed a question mark from “Yes”', 'Chelsea ', 'Black High Top Converse ',
        'Dr Martin boots', 'Dr Martina', 'Doc martins', 'Ugh', 'Ugg red boot']

In [29]:
m.threshold = 80
r = m.run(queries=queries, limit=3)
pprint(r)

{'2 dragon ball z t shirts': [('rainbow', 51), ('jan sport', 48), ('roxy', 45)],
 '2976 Chelsea ': [('heelys', 60), ('clarks', 45), ('converse', 45)],
 '3/3': [('dr. martens', 30), ('jan sport', 30), ("journey's", 30)],
 'AS OX OCEAN BLISS\n&\nSOFTEY PNK': [('jan sport', 48),
                                      ("levi's", 45),
                                      ('nike', 45)],
 'Black High Top Converse ': [('converse', 90),
                              ('new balance', 55),
                              ('chaco', 51)],
 'Chelsea ': [('heelys', 62), ('converse', 53), ('herschel', 53)],
 'Doc martins': [('dr. martens', 73), ('clarks', 47), ('adidas', 45)],
 'Dr Martin boots': [('dr. martens', 69), ('rainbow', 64), ('dc', 45)],
 'Dr Martina': [('dr. martens', 76), ('rainbow', 47), ('madden girl', 46)],
 'Hydro flask': [('fila', 68), ('clarks', 45), ('dc', 45)],
 'Removed a question mark from “Yes”': [('birkenstock', 47),
                                        ('clarks', 45),
        

In [11]:
m.run(queries, limit=3)

{'Rick and Morty T-shirt and socks': [('jan sport', 60),
  ('crocs', 54),
  ('birkenstock', 50)],
 'test': [('birkenstock', 68), ('dr. martens', 68), ('teva', 50)],
 'mia': [('adidas', 60), ('champion', 60), ('dr. martens', 60)],
 'Shoe cleaner ': [('sorel', 54), ('converse', 53), ('herschel', 53)],
 "Vans. They weren't in stock so I ordered and shipped to my house ": [('vans',
   60),
  ('birkenstock', 58),
  ('dr. martens', 50)],
 'Removed a question mark from “Yes”': [('birkenstock', 47),
  ('clarks', 45),
  ('reebok', 45)],
 'Chelsea ': [('heelys', 62), ('converse', 53), ('herschel', 53)],
 'Black High Top Converse ': [('converse', 90),
  ('new balance', 55),
  ('chaco', 51)]}

In [12]:
m.

0.8

In [56]:
def match(patterns, strings_to_match, threshold=80, verbose=False):
    matched = [process.extractOne(str(i), patterns) for i in strings_to_match]
    #if verbose:
    matched = [t[0] if t[1]>=threshold else 'no_match' for t in matched]
    return matched    

In [ ]:
def match(patterns, strings_to_match, threshold=80, verbose=False):
    def func(s, patterns):
        return process.extractOne(str(s), patterns)
    matched = {i:(func(i, patterns)[0] if func(i, patterns)[1]>threshold else 'no_match') for i in strings_to_match}
    #matched = [t[0] if t[1]>=threshold else 'no_match' for t in matched]
    #if verbose:
    #    return matched
    return matched

In [57]:
%%time

matched = match(patterns=patterns, strings_to_match=strings_to_match)

CPU times: user 10.7 s, sys: 0 ns, total: 10.7 s
Wall time: 10.7 s


In [55]:
matched

{'Uggs': 'uggs',
 'Adidas ': 'adidas',
 'Timberland': 'timberlands',
 'Converse': 'converse',
 'Red tims ': 'no_match',
 'Vans': 'vans',
 'Vans ': 'vans',
 'Converse ': 'converse',
 'Adidas': 'adidas',
 'Converse and Sperry ': 'converse',
 'Reebok ': 'reebok',
 'Addias and Vans ': 'vans',
 'Doc Martens ': 'dr. martens',
 'Perry and ugg': 'perry',
 'Hey dude': 'dude',
 'Converse and uggs': 'converse',
 'Ugg ': 'uggs',
 'Yes': 'no_match',
 'Adidas and hey dude': 'adidas',
 'Ugg': 'uggs',
 'Convers ': 'converse',
 'vans': 'vans',
 'converse ': 'converse',
 "Levi's ": "levi's",
 'Filas': 'fila',
 'Dr marten': 'dr. martens',
 'Uggs ': 'uggs',
 'test': 'no_match',
 'uggs/dr martens ': 'dr. martens',
 "timberland's ": 'timberlands',
 'Classic Reebok': 'reebok',
 'Vans and FILA': 'fila',
 'Rick and Morty T-shirt and socks': 'no_match',
 'Lacoste timberlands ': 'lacoste',
 'Nike ': 'nike',
 'Vans, puma': 'puma',
 'mia': 'no_match',
 'Premium Wheat Timberlands': 'timberlands',
 'Shoe cleaner ': 

In [43]:
df_31665.head()

,question_id,reply_id,created_at,question,reply,matched
0,31665,18475753,2020-11-27 18:34:35,Tell us...what brand(s) did you purchase?,Uggs,uggs
1,31665,21873401,2021-02-05 17:24:45,Tell us...what brand(s) did you purchase?,Adidas,adidas
2,31665,21871152,2021-02-05 16:43:52,Tell us...what brand(s) did you purchase?,Timberland,timberlands
3,31665,21873596,2021-02-05 17:28:04,Tell us...what brand(s) did you purchase?,Converse,converse
4,31665,21871473,2021-02-05 16:52:05,Tell us...what brand(s) did you purchase?,Red tims,no_match


In [44]:
df_31665.loc[:, 'matched'] = matched
df_31665.head()

/home/george/anaconda3/envs/frm/lib/python3.9/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


,question_id,reply_id,created_at,question,reply,matched
0,31665,18475753,2020-11-27 18:34:35,Tell us...what brand(s) did you purchase?,Uggs,uggs
1,31665,21873401,2021-02-05 17:24:45,Tell us...what brand(s) did you purchase?,Adidas,adidas
2,31665,21871152,2021-02-05 16:43:52,Tell us...what brand(s) did you purchase?,Timberland,timberlands
3,31665,21873596,2021-02-05 17:28:04,Tell us...what brand(s) did you purchase?,Converse,converse
4,31665,21871473,2021-02-05 16:52:05,Tell us...what brand(s) did you purchase?,Red tims,no_match


In [45]:
(df_31665.matched=="no_match").mean()

0.09309167941626456

In [48]:
no_matches = df_31665.query('matched=="no_match"').sample(50).reply.values.tolist()
no_matches

['Doc martins ',
 'Crep Protect',
 'Shirt wallet socks ',
 'Mia ',
 'Socks ',
 'Doc Maarten ',
 'Docs ',
 'Ugh',
 'Some DR. Martins',
 'Aitwait',
 'Friends sweatshirt in black',
 'Docs',
 'Dock martens ',
 'Vanes, dr Martin ',
 'Naruto sweat shirt',
 'Doc Martns',
 'Doc martins ',
 'Socks and shoe laces',
 'Foxy',
 'Dragon ball z tee',
 'Dirty Laundry ',
 'DocMartens',
 'Two pairs of shoes',
 'Ugh',
 'AS OX OCEAN BLISS\n&\nSOFTEY PNK',
 'Tommy shoes',
 'Bio world ',
 'Sanuk ',
 'Ge as',
 'The original doc martins ',
 'Doc Martin ',
 'Hershel wallets',
 'Yes',
 'Addis is',
 'I purchased UGG boots ',
 'Ugh',
 'Docs',
 'Doc Martin ',
 'Ugh',
 'Ugh',
 'Wesbreak',
 'Doc Martin ',
 'Doc marteind',
 'bucket hat',
 'Ugh',
 'Tiger',
 'Chucks',
 'Martins ',
 'Hydro flask',
 'Ugh']

In [49]:
match(patterns, no_matches, include_score=True)

[('dr. martens', 73),
 ('crocs', 47),
 ("levi's", 60),
 ('dr. martens', 60),
 ('birkenstock', 72),
 ('dr. martens', 73),
 ('crocs', 67),
 ('uggs', 57),
 ('dr. martens', 74),
 ('adidas', 46),
 ('new balance', 50),
 ('crocs', 67),
 ('dr. martens', 78),
 ('vans', 68),
 ('toms', 68),
 ('dr. martens', 76),
 ('dr. martens', 73),
 ('sorel', 54),
 ('roxy', 75),
 ('teva', 51),
 ('timberlands', 50),
 ('dr. martens', 76),
 ('sorel', 54),
 ('uggs', 57),
 ('jan sport', 48),
 ('toms', 68),
 ('sorel', 54),
 ('vans', 44),
 ("levi's", 55),
 ('dr. martens', 66),
 ('dr. martens', 67),
 ('herschel', 79),
 ('converse', 60),
 ('adidas', 57),
 ('uggs', 68),
 ('uggs', 57),
 ('crocs', 67),
 ('dr. martens', 67),
 ('uggs', 57),
 ('uggs', 57),
 ('sorel', 54),
 ('dr. martens', 67),
 ('dr. martens', 70),
 ('dude', 45),
 ('uggs', 57),
 ('tommy hilfiger', 72),
 ('crocs', 55),
 ('dr. martens', 77),
 ('fila', 68),
 ('uggs', 57)]

In [16]:
str2Match = "apple inc"
strOptions = ["Apple Inc.","apple park","apple incorporated","iphone"]

process.extract(str2Match,strOptions)

[('Apple Inc.', 100),
 ('apple incorporated', 90),
 ('apple park', 67),
 ('iphone', 30)]

In [23]:
process.extract('Dr. Martens', patterns)

[('doc marten', 76),
 ('reefs', 54),
 ('madden girl', 54),
 ('ralph lauren', 52),
 ('vans', 45)]

In [13]:
Str1 = "The supreme court case of Nixon vs The United States"
Str2 = "Nixon v. United States"
Ratio = fuzz.ratio(Str1.lower(),Str2.lower())
Partial_Ratio = fuzz.partial_ratio(Str1.lower(),Str2.lower())
Token_Sort_Ratio = fuzz.token_sort_ratio(Str1,Str2)
Token_Set_Ratio = fuzz.token_set_ratio(Str1,Str2)
print(Ratio)
print(Partial_Ratio)
print(Token_Sort_Ratio)
print(Token_Set_Ratio)

57
77
58
95
